In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load data:
generation_data_1 = pd.read_csv("data/raw/Plant_1_Generation_Data.csv")
generation_data_2 = pd.read_csv("data/raw/Plant_2_Generation_Data.csv")
weather_data_1 = pd.read_csv("data/raw/Plant_1_Weather_Sensor_Data.csv")
weather_data_2 = pd.read_csv("data/raw/Plant_2_Weather_Sensor_Data.csv")

In [4]:
# Format timestamp:
generation_data_1["DATE_TIME"] = pd.to_datetime(generation_data_1.DATE_TIME)
generation_data_2["DATE_TIME"] = pd.to_datetime(generation_data_2.DATE_TIME)
weather_data_1["DATE_TIME"] = pd.to_datetime(weather_data_1.DATE_TIME)
weather_data_2["DATE_TIME"] = pd.to_datetime(weather_data_2.DATE_TIME)

C:\Users\nmert\AppData\Local\Temp\ipykernel_16048\4217505130.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  generation_data_1["DATE_TIME"] = pd.to_datetime(generation_data_1.DATE_TIME)


In [5]:
generation_data_1["SOURCE_KEY"] = generation_data_1["SOURCE_KEY"].astype("category")
generation_data_2["SOURCE_KEY"] = generation_data_2["SOURCE_KEY"].astype("category")
weather_data_1["SOURCE_KEY"] = weather_data_1["SOURCE_KEY"].astype("category")
weather_data_2["SOURCE_KEY"] = weather_data_2["SOURCE_KEY"].astype("category")

In [6]:
# Combine:
plant_1 = pd.merge(generation_data_1, weather_data_1.drop(columns=["PLANT_ID", "SOURCE_KEY"]), on = "DATE_TIME")
plant_2 = pd.merge(generation_data_2, weather_data_2.drop(columns=["PLANT_ID",  "SOURCE_KEY"]), on = "DATE_TIME")
data = pd.concat([plant_1, plant_2])

In [7]:
# Cast data types:
data["PLANT_ID"] = data["PLANT_ID"].astype("category")

In [8]:
# Rename columns
mapping = {
  "PLANT_ID": "plantId",
  "DATE_TIME": "dtm",
  "SOURCE_KEY": "sourceKey",
  "DC_POWER": "pow_dc",
  "AC_POWER": "pow_ac",
  "DAILY_YIELD": "yield_daily",
  "TOTAL_YIELD": "yield_total",
  "AMBIENT_TEMPERATURE": "temp_ambient",
  "MODULE_TEMPERATURE": "temp_module",
  "IRRADIATION": "irradiation"
}

data = data.rename(columns = mapping)

In [9]:
# Drop duplicate rows:
data = data.drop_duplicates()

In [10]:
# Calculate local time:
data["time_local"] = data["dtm"].dt.hour + data["dtm"].dt.minute / 60

In [11]:
# Sort:
data = data.sort_values(["plantId", "sourceKey", "dtm"])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136472 entries, 0 to 67697
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   dtm           136472 non-null  datetime64[ns]
 1   plantId       136472 non-null  category      
 2   sourceKey     136472 non-null  object        
 3   pow_dc        136472 non-null  float64       
 4   pow_ac        136472 non-null  float64       
 5   yield_daily   136472 non-null  float64       
 6   yield_total   136472 non-null  float64       
 7   temp_ambient  136472 non-null  float64       
 8   temp_module   136472 non-null  float64       
 9   irradiation   136472 non-null  float64       
 10  time_local    136472 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(8), object(1)
memory usage: 11.6+ MB


In [13]:
data.head()

,dtm,plantId,sourceKey,pow_dc,pow_ac,yield_daily,yield_total,temp_ambient,temp_module,irradiation,time_local
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,25.184316,22.857507,0.0,0.00
21,2020-05-15 00:15:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,25.084589,22.761668,0.0,0.25
42,2020-05-15 00:30:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,24.935753,22.592306,0.0,0.50
63,2020-05-15 00:45:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,24.846130,22.360852,0.0,0.75
84,2020-05-15 01:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,24.621525,22.165423,0.0,1.00


In [18]:
# Save preprocessed data:
data.to_parquet("data/solar_generation.parquet", index = False)